# --- Day 3: Gear Ratios ---
https://adventofcode.com/2023/day/3

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. **What is the sum of all of the part numbers in the engine schematic?**

In [150]:
import numpy as np
import re

# init a data array to hold input data
data = []
scores = []

# gi
with open("input.txt", "r") as f:
    lines = f.read().splitlines()
    for line in lines:
        data.append([x for x in line])

# transform it into a 2d array numpy array and check the amount of rows and cols
# [rows, columns]
data = np.array(data)
nrRows = data.shape[0]
nrCols = data.shape[1]

# This function will check if a number is adjacent to a symbol based on its coords
def symbol_adjacent(digit_coord):
    minRow = digit_coord[0]-1
    maxRow = digit_coord[0]+2 # +2 because the range is not inclusive
    minCol = digit_coord[1]-1
    maxCol = digit_coord[1]+2 # +2 because the range is not inclusive
    if minRow < 0:
        minRow = 0
    if maxRow > nrRows:
        maxRow = nrRows-1
    if minCol < 0:
        minCol = 0

    
    # to know if it's adjacent we need to grab a block (3x3 for most) with the coord in the center and check for special chars
    chunk = data[minRow:maxRow,minCol:maxCol]
    # we then flatten that chunk into a string
    chunkString = "".join(chunk.flatten())
    # and search for any specials char, if found return True
    if re.search(r"[^\d\.\n]",chunkString):
        return True
    return False

# init some vars to keep track
adjacent = False
track = ""

def reset():
    global adjacent, track
    adjacent = False
    track = ""

# looping through every cell
for i in range(nrRows):
    # non digit char found, we 
    if adjacent is True:
        scores.append(int(track))
    reset()
    for j in range(nrCols):
        char = data[i,j]
        # if the char is a digit
        if char.isdigit():
            # add it to the tracking string
            track += char
            # check if its adjacent to a symbol and if so set the bool to True
            if symbol_adjacent([i,j]):
                adjacent = True
        else:
            # non digit char found, we 
            if adjacent is True:
                scores.append(int(track))
            reset()
        
print("The answer to part 1 is:",sum(scores))
    

The answer to part 1 is: 535078


## --- Part Two ---
The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

**What is the sum of all of the gear ratios in your engine schematic?**



In [249]:
import numpy as np
import re
import math

# init a data array to hold input data
data = []
scores = []

# gi
with open("input.txt", "r") as f:
    lines = f.read().splitlines()
    for line in lines:
        data.append([x for x in line])

# transform it into a 2d array numpy array and check the amount of rows and cols
# [rows, columns]
data = np.array(data)
nrRows = data.shape[0]
nrCols = data.shape[1]

# a list of all the coords of asteriks (*) in the data
astCoords = []

# looping through every cell and collecting the coords of all the gears (*)
for i in range(nrRows):
    for j in range(nrCols):
        char = data[i,j]
        if re.search(r"[\*]",char):
            astCoords.append([i,j])

# Reworked the function to reutrn the coords of the nearby asterix
def symbol_adjacent(digit_coord):
    for x in astCoords:
        # check based on the euclidian distance between points, if its <2 its in range
        if math.dist(digit_coord, x) < 2:
            return x
    return []

# init some vars to keep track
adjacents = []
track = ""

# so the idea is to keep a list of all asterix coords and add nearby scores to the list
overall = dict()

def reset():
    global adjacents, track
    adjacents = []
    track = ""

# looping through every cell
for i in range(nrRows):
    for j in range(nrCols):
        char = data[i,j]
        # if the char is a digit
        if char.isdigit():
            # add it to the tracking string
            track += char
            # check if its adjacent to a symbol
            nearby = symbol_adjacent([i,j])
            # if there are more than 0 nearby , and non-duplicate, 
            if(len(nearby) > 0 and nearby not in adjacents):
                adjacents.append(nearby)
        else:
            # end of digits, check if the previous adjacents are around
            if len(adjacents) > 0:
                for x in adjacents:
                    # create a unique key if it doesnt exist and init a list []
                    if str(x) not in overall:
                        overall[str(x)] = []
                    # append score to the asterix coord
                    overall[str(x)].append(track)
            reset()
# print(overall)

# loop through all keys in the overall results
for key in overall:
    # only applicable if there are 2 or more
    if len(overall[key])>1:
        # convert scores to int and mulitply them and add to overall scores
        score = [int(x) for x in overall[key]]
        scores.append(np.prod(score))

# print sum of all scores
print("The answer to part 2 is:",sum(scores))
    

The answer to part 2 is: 75312571
